In [ ]:
import requests
import csv
import pandas as pd
import json
import ast

### Tyres - Stint

In [ ]:
from matplotlib import pyplot as plt

import fastf1
import fastf1.plotting


session = fastf1.get_session(2022, 'Hungary', 'R')
session.load()
laps = session.laps

drivers = session.drivers

drivers = [session.get_driver(driver)["Abbreviation"] for driver in drivers]

results = session.results
positions = results[["DriverNumber", "GridPosition", "Position"]]

stints = laps[["Driver", "DriverNumber", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "DriverNumber", "Stint", "Compound"])
stints = stints.count().reset_index()

stints = stints.rename(columns={"LapNumber": "StintLength"})


In [ ]:
stints_with_position = pd.merge(stints, positions, on=['DriverNumber'], how='left')
stints_with_position = stints_with_position.astype({'Position':'int64', 'GridPosition':'int64'})
stints_with_position = stints_with_position.sort_values(by=['Position', 'Stint'])
stints_with_position.head()

In [ ]:
import plotly.graph_objects as go
import numpy as np
# Crear una figura en Plotly
fig = go.Figure()

compounds_set = set()
# Iterar por cada piloto
for driver in drivers:

    driver_stints = stints_with_position.loc[stints["Driver"] == driver].copy()

    driver_stints[['Stint', 'Compound']] = driver_stints[['Stint', 'Compound']].replace('nan', np.nan)
    driver_stints[['Stint', 'Compound']] = driver_stints[['Stint', 'Compound']].bfill()
    
    previous_stint_end = 0
    for _, row in driver_stints.iterrows():
        # Obtener el color correspondiente al compuesto
        compound_color = fastf1.plotting.get_compound_color(row["Compound"], session=session)
        compounds_set.add((row["Compound"],compound_color))

        # Añadir una barra horizontal para cada stint del piloto
        fig.add_trace(go.Bar(
            y=[driver],  # Eje Y muestra al piloto
            x=[row["StintLength"]],  # Eje X es la duración del stint
            base=previous_stint_end,  # El inicio de la barra
            orientation='h',  # Barras horizontales
            marker=dict(color=compound_color),  # Colores y bordes
            name=row["Compound"],  # Nombre del compuesto para el hover,
            hovertemplate=(
                f"Piloto: {driver}<br>" +
                f"Compuesto: {row['Compound']}<br>" +
                f"Duración del stint: {row['StintLength']} vueltas<br>" +
                f"Inicia en la vuelta: {previous_stint_end}<br>"+
                f"Posicion inicial: {row['GridPosition']}<br>"+
                f"Posicion final: {row['Position']}<br>"
            ),
            showlegend=False 
        ))

        previous_stint_end += row["StintLength"]
    
for compound, color in compounds_set:
    fig.add_trace(go.Bar(
        y=[None],  # Para no mostrar una barra visible, solo usar la leyenda
        x=[0],  # Valor de 0 para que no se dibuje una barra visible
        name=compound,  # Nombre de la entrada de la leyenda
        marker=dict(color=color),  # El color correspondiente al compuesto
        showlegend=True  # Mostrar una entrada en la leyenda para cada compuesto
))

# Configuración del diseño
fig.update_layout(
    title="2022 Hungarian Grand Prix Strategies",
    height=800,
    xaxis_title="Lap Number",
    yaxis_title="Driver",
    barmode='stack',  # Las barras se apilan horizontalmente
    xaxis=dict(showgrid=False, zeroline=False),  # Ocultar líneas de rejilla
    yaxis=dict(autorange="reversed"),  # Invertir el orden de los pilotos
    template='plotly_white'  # Tema blanco limpio
)

# Mostrar el gráfico
fig.show()


In [ ]:
laps.tail()

In [ ]:
laps_updated = laps.drop(columns=['Time', 'PitOutTime', 'PitInTime', 'FastF1Generated', 'FreshTyre', 'IsAccurate', 'Deleted', 'DeletedReason'], axis=1)

In [ ]:
import plotly.express as px

# Plotting using Plotly Express
fig = px.line(laps_updated, 
            x='LapNumber', 
            y='Position', 
            color='Driver', 
            markers=True)

# Mostrar el gráfico
fig.show()
